In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
df= pd.read_excel('spacestri.xlsx')
print(df.head())

In [13]:
import docx  # python-docx est importé en tant que 'docx'
import pandas as pd

def extraire_donnees_depuis_docx(chemin_docx):
    document = docx.Document(chemin_docx)
    
    donnees = []
    nom_actuel = None
    question_actuelle = None
    questions_reponses = {}
    
    for paragraphe in document.paragraphs:
        texte = paragraphe.text.strip()
        
        if paragraphe.style.name.startswith('Heading'):  # Suppose que le nom est un titre
            if nom_actuel and questions_reponses:
                donnees.append({'nom': nom_actuel, 'questions_reponses': questions_reponses})
            nom_actuel = texte
            questions_reponses = {}
            question_actuelle = None
        
        elif texte.startswith("Q"):  
            question_actuelle = texte
            questions_reponses[question_actuelle] = ''
        
        elif question_actuelle:  # Suppose que tout texte après une question est une réponse
            questions_reponses[question_actuelle] += ' ' + texte
    
    if nom_actuel and questions_reponses:
        donnees.append({'nom': nom_actuel, 'questions_reponses': questions_reponses})
    
    return donnees

# Utilisation
chemin_docx = 'réponses-ARS-Carré-dart-sans mise en fome.docx'
donnees_extraites = extraire_donnees_depuis_docx(chemin_docx)

# Charger le fichier Excel avec pandas
df = pd.read_excel('spacestri.xlsx')

# Ajouter les colonnes pour les questions et réponses dans le DataFrame
df['question1'] = ''
df['réponse1'] = ''
df['question2'] = ''
df['réponse2'] = ''

# Vérifier la dernière valeur d'ID dans le DataFrame
dernier_id = df['id'].max() if 'id' in df.columns else 0  # Si la colonne 'id' existe, on prend sa valeur max, sinon on commence à 0


# Parcourir les données extraites du DOCX
for entree in donnees_extraites:
    nom = entree['nom']
    questions = list(entree['questions_reponses'].keys())
    reponses = list(entree['questions_reponses'].values())
    
    # Si le nom est dans le DataFrame, on remplit les colonnes correspondantes
    if nom in df['nom'].values:
        df.loc[df['nom'] == nom, 'question1'] = questions[0] if len(questions) > 0 else ''
        df.loc[df['nom'] == nom, 'réponse1'] = reponses[0] if len(reponses) > 0 else ''
        df.loc[df['nom'] == nom, 'question2'] = questions[1] if len(questions) > 1 else ''
        df.loc[df['nom'] == nom, 'réponse2'] = reponses[1] if len(reponses) > 1 else ''
    
    # Si le nom n'est pas dans le DataFrame, on ajoute une nouvelle ligne
    else:
        dernier_id +=1 
        nouvelle_ligne = pd.DataFrame({
            'id': [dernier_id],
            'nom': [nom],
            'question1': [questions[0] if len(questions) > 0 else ''],
            'réponse1': [reponses[0] if len(reponses) > 0 else ''],
            'question2': [questions[1] if len(questions) > 1 else ''],
            'réponse2': [reponses[1] if len(reponses) > 1 else '']
        })
        df = pd.concat([df, nouvelle_ligne], ignore_index=True)

# Sauvegarder le DataFrame mis à jour dans un fichier Excel ou CSV
df.to_excel('fichier_mis_a_jour.xlsx', index=False)
print("Mise à jour du fichier Excel terminée et sauvegardée dans 'fichier_mis_a_jour.xlsx'")


Mise à jour du fichier Excel terminée et sauvegardée dans 'fichier_mis_a_jour.xlsx'


In [ ]:
import docx  # python-docx est importé en tant que 'docx'
import pandas as pd
import re  # Pour utiliser les expressions régulières

def extraire_donnees_depuis_docx(chemin_docx):
    document = docx.Document(chemin_docx)
    
    donnees = []
    nom_actuel = None
    question_actuelle = None
    questions_reponses = {}
    
    for paragraphe in document.paragraphs:
        texte = paragraphe.text.strip()
        
        if paragraphe.style.name.startswith('Heading'):  # Suppose que le nom est un titre
            if nom_actuel and questions_reponses:
                donnees.append({'nom': nom_actuel, 'questions_reponses': questions_reponses})
            nom_actuel = texte
            questions_reponses = {}
            question_actuelle = None
        
        elif texte.startswith("Q"):  # Suppose que les questions commencent par "Q"
            question_actuelle = texte
            questions_reponses[question_actuelle] = ''
        
        elif question_actuelle:  # Suppose que tout texte après une question est une réponse
            questions_reponses[question_actuelle] += ' ' + texte
    
    if nom_actuel and questions_reponses:
        donnees.append({'nom': nom_actuel, 'questions_reponses': questions_reponses})
    
    return donnees

def extraire_nom_et_sites(texte):
    # Regex améliorée pour capturer des noms composés et gérer mieux les titres
    match_nom = re.search(r'([A-Z][a-zéèêîï]+(?:\s[A-Z][a-zéèêîï]+)+)', texte)
    nom_responsable = match_nom.group(0) if match_nom else 'Nom non trouvé'
    
    # Regex pour capturer tous les sites web dans le texte (peut en avoir plusieurs)
    sites_web = re.findall(r'(www\.[\w.-]+|@\w+)', texte)
    if not sites_web:
        sites_web = ['Site web non trouvé']
    
    return nom_responsable, sites_web

# Parcourir les données extraites du DOCX
for entree in donnees_extraites:
    nom = entree['nom']
    questions = list(entree['questions_reponses'].keys())
    reponses = list(entree['questions_reponses'].values())
    
    # Si le nom est dans le DataFrame, on remplit les colonnes correspondantes
    if nom in df['nom'].values:
        df.loc[df['nom'] == nom, 'question1'] = questions[0] if len(questions) > 0 else ''
        df.loc[df['nom'] == nom, 'réponse1'] = reponses[0] if len(reponses) > 0 else ''
        df.loc[df['nom'] == nom, 'question2'] = questions[1] if len(questions) > 1 else ''
        df.loc[df['nom'] == nom, 'réponse2'] = reponses[1] if len(reponses) > 1 else ''
        
        # Extraire le nom du responsable et les sites web à partir de la réponse 2
        nom_responsable, sites_web = extraire_nom_et_sites(reponses[1] if len(reponses) > 1 else '')
        
        # Afficher pour vérification
        print(f"Nom : {nom}")
        print(f"Nom du responsable : {nom_responsable}")
        print(f"Sites web : {', '.join(sites_web)}\n")
        
        df.loc[df['nom'] == nom, 'nom_responsable'] = nom_responsable
        df.loc[df['nom'] == nom, 'site_web'] = ', '.join(sites_web)
    
    # Si le nom n'est pas dans le DataFrame, on ajoute une nouvelle ligne
    else:
        nom_responsable, sites_web = extraire_nom_et_sites(reponses[1] if len(reponses) > 1 else '')
        
        # Afficher pour vérification
        print(f"Nom : {nom}")
        print(f"Nom du responsable : {nom_responsable}")
        print(f"Sites web : {', '.join(sites_web)}\n")
        
        nouvelle_ligne = pd.DataFrame({
            'nom': [nom],
            'question1': [questions[0] if len(questions) > 0 else ''],
            'réponse1': [reponses[0] if len(reponses) > 0 else ''],
            'question2': [questions[1] if len(questions) > 1 else ''],
            'réponse2': [reponses[1] if len(reponses) > 1 else ''],
            'nom_responsable': [nom_responsable],
            'site_web': [', '.join(sites_web)]
        })
        df = pd.concat([df, nouvelle_ligne], ignore_index=True)




# Sqlite

In [2]:
import pandas as pd

# Charger le fichier excel
df= pd.read_excel('fichier_mis_a_jour.xlsx')

In [ ]:
print(df.head())

# Exemple de filtrage
#filtered_df = df[df['colonne'] == 'valeur']


In [5]:
print(df.columns)

Index(['id', 'nom', 'date_ouverture', 'date_fermeture', 'website', 'email',
       'adresse', 'horaires', 'presentation', 'responsables', 'historique',
       'activites', 'expositions', 'publications', 'events', 'residences',
       'workshop', 'archive', 'librairie', 'coworking', 'education',
       'nb_expos_an', 'nb_artistes_an', 'partenaires', 'created_at',
       'updated_at', 'slug', 'pays', 'ville', 'latitude', 'longitude',
       'adresse_complete', 'codepostal', 'telephone', 'visite', 'user_id',
       'question1', 'réponse1', 'question2', 'réponse2'],
      dtype='object')


In [7]:
df_selection1 = df[['id', 'nom', 'historique', 'activites', 'presentation', 
                    'date_ouverture', 'date_fermeture', 'website', 'pays', 
                    'ville', 'latitude', 'longitude']]
# Vérifier le nombre de valeurs manquantes dans la colonne 'id'
missing_values_count = df_selection1['id'].isnull().sum()
print(f"Nombre de valeurs manquantes dans 'id' : {missing_values_count}")

Nombre de valeurs manquantes dans 'id' : 4


In [ ]:
# Vérifier les valeurs uniques dans la colonne 'id'
duplicates = df_selection1['id'][df_selection1['id'].duplicated(keep=False)]
if not duplicates.empty:
    print("Les identifiants suivants sont en double :")
    print(duplicates)
else:
    print("Tous les identifiants sont uniques.")


In [4]:
import sqlite3
import pandas as pd

# Charger le fichier Excel dans un DataFrame
df = pd.read_excel('fichier_mis_a_jour.xlsx')

# Sélectionner certaines colonnes du DataFrame
df_selection1 = df[['id', 'nom', 'historique', 'activites', 'presentation', 
                    'date_ouverture', 'date_fermeture', 'website', 'pays', 
                    'ville', 'latitude', 'longitude']]

df_selection2 = df[['id', 'responsables']]
df_selection3 = df[['id', 'question1', 'réponse1', 'question2', 'réponse2']]

# Créer/établir une connexion à une base de données SQLite
with sqlite3.connect('spacestri.db', timeout=10) as conn:
    cursor = conn.cursor()

# Créer la table Question_Reponse
cursor.execute('''
CREATE TABLE IF NOT EXISTS Question_Reponse (
    id_QR INTEGER PRIMARY KEY,
    question1 TEXT,
    reponse1 TEXT,
    question2 TEXT,
    reponse2 TEXT
);
''')

# Créer la table Responsable
cursor.execute('''
CREATE TABLE IF NOT EXISTS Responsable (
    id_R INTEGER PRIMARY KEY,
    responsables TEXT,
    id_QReponse INTEGER,         
    FOREIGN KEY (id_QReponse) REFERENCES Question_Reponse (id_QR)         
);
''')

# Créer la table Espace
cursor.execute('''
CREATE TABLE IF NOT EXISTS Espace (
    id_E INTEGER PRIMARY KEY,
    nom TEXT,
    historique TEXT,
    activites TEXT,
    presentation TEXT,
    date_ouverture DATE,
    date_fermeture DATE,
    website TEXT,
    pays TEXT,
    ville TEXT,
    latitude TEXT,
    longitude TEXT
);
''')

# Créer la table de liaison 'tenir' pour relier Espace et Responsable
cursor.execute('''
CREATE TABLE IF NOT EXISTS tenir (
    id_tenir INTEGER PRIMARY KEY,
    id_espace INTEGER,
    id_responsable INTEGER,
    FOREIGN KEY (id_espace) REFERENCES Espace(id_E),
    FOREIGN KEY (id_responsable) REFERENCES Responsable(id_R)
);
''')

# Insérer les données dans la table 'Espace'
for i, row in df_selection1.iterrows():
    cursor.execute('''
    INSERT INTO Espace (id_E, nom, historique, activites, presentation, 
                        date_ouverture, date_fermeture, website, pays, 
                        ville, latitude, longitude) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['id'], row['nom'], row['historique'], row['activites'], row['presentation'], 
          row['date_ouverture'], row['date_fermeture'], row['website'], row['pays'], 
          row['ville'], row['latitude'], row['longitude']))

    if i % 100 == 0:  # Effectuer un commit tous les 100 enregistrements
        conn.commit()

# N'oubliez pas de faire un dernier commit après la boucle
conn.commit()

# Insérer les données dans la table 'Question_Reponse'
for i, row in df_selection3.iterrows():
    cursor.execute('''
    INSERT INTO Question_Reponse (id_QR, question1, reponse1, question2, reponse2)
    VALUES (?, ?, ?, ?, ?)
    ''', (row['id'], row['question1'], row['réponse1'], row['question2'], row['réponse2']))

    if i % 100 == 0:  # Effectuer un commit tous les 100 enregistrements
        conn.commit()

# Un dernier commit après la boucle
conn.commit()

# Insérer les données dans la table 'Responsable'
for i, row in df_selection2.iterrows():
    cursor.execute('''
    INSERT INTO Responsable (id_R, responsables, id_QReponse)
    VALUES (?, ?, ?)
    ''', (row['id'], row['responsables'], row['id']))

    if i % 100 == 0:  # Effectuer un commit tous les 100 enregistrements
        conn.commit()

# Un dernier commit après la boucle
conn.commit()


id_tenir = 1  # Initialiser un compteur pour id_tenir

for i, row in df_selection1.iterrows():
    cursor.execute('''
    INSERT INTO tenir (id_tenir, id_espace, id_responsable)
    VALUES (?, ?, ?)
    ''', (id_tenir, row['id'], row['id']))  # Utiliser l'id_tenir incrémenté

    id_tenir += 1  # Incrémenter l'ID pour la prochaine ligne

    if i % 100 == 0:  # Effectuer un commit tous les 100 enregistrements
        conn.commit()


# Un dernier commit après la boucle
conn.commit()

# Sauvegarder (commit) les changements
conn.commit()

# Fermer la connexion à la base de données
conn.close()


OperationalError: database is locked